## Steam Reviews Get!

This will act as a live document as I use it to source, clean and analyze a dataset sourced from Steam. 


#### Objectives
* Source a dataset of all English reviews on steam!
    * Access publicly available api calls for store.steampowered
    * end with a dataframe for which a row will hold the app title/id, review author details, and review details
* Write and save dataset to Kaggle for convenience
* Thoroughly document process to help learn and also later reference as a tutorial

#### Concerns
* Avoiding DOS
    * I'm making an obscene number of requests from Valve what happens if I request too much?
    * Can I limit the amount of requests I make and request in a piecemeal fashion?
        * From research, it looks like steam limits the amount of requests allowed to 100,000 per calendar day. 
        * Approach two different ways: sleep an amount every request to ensure I don't go over (865ms per request) or count requests via a global variable and sleep on 100,000
        

In [92]:
import pandas as pd

In [59]:
import requests

URL= "https://store.steampowered.com/appreviews/1533420?json=1"
cursor = '*'
PARAMS = {
    'language':'english',
    'cursor': cursor,
    'num_per_page' : 1,
}

1533420
r = requests.get(URL, PARAMS)
r=r.json()

In [85]:
# print(r)

print(r["query_summary"]['total_reviews'])
print(r["cursor"])

1265
AoIIP3ADBH73874D


In [94]:
# print(r["reviews"])
df = pd.json_normalize(r["reviews"])
# df.head()

In [82]:
def getAppReviews(app, step):
    URL = "https://store.steampowered.com/appreviews/" + str(app) +"?json=1"

    tally=step
    PARAMS = {
        'language':'english',
        'cursor': '*', #initial value defined in steam's documentation
        'num_per_page' : step,
    }

    r = requests.get(URL, PARAMS)
    r=r.json()
    nCursor = {'cursor' : r["cursor"]}
    PARAMS.update(nCursor)
    total = r["query_summary"]['total_reviews']
    df = pd.json_normalize(r["reviews"])
    while tally < total:
        r = requests.get(URL, PARAMS)
        r=r.json()
        df = pd.concat([df, (pd.json_normalize(r["reviews"]))])
        tally += step
        nCursor = {'cursor' : r["cursor"]}
        PARAMS.update(nCursor)
    return df
    

In [90]:
df = getAppReviews(1533420, 50)

In [91]:
df.shape

(1246, 22)

We now have a method to get reviews given a specific app. Next we get a list of all apps to call from.